# Module 3 - LLM Fundamentals Assessment

**TEST SUBMISSION - For grading pipeline testing**

In [ ]:
# ===== LLM GATEWAY CONFIGURATION =====
LLM_BASE_URL = "http://localhost:11434"
LLM_API_KEY = None
DEFAULT_MODEL = "phi3:mini"

import requests
import json
import re

In [ ]:
# Exercise 1: Implement call_llm function

def call_llm(prompt: str, temperature: float = 0.0) -> str:
    """Call an LLM endpoint and return the response text."""
    headers = {
        "Content-Type": "application/json",
        "ngrok-skip-browser-warning": "true",
        "Bypass-Tunnel-Reminder": "true",
    }
    
    if LLM_API_KEY:
        headers["X-API-Key"] = LLM_API_KEY
        endpoint = f"{LLM_BASE_URL.rstrip('/')}/chat/direct"
    else:
        endpoint = f"{LLM_BASE_URL.rstrip('/')}/api/chat"
    
    payload = {
        "model": DEFAULT_MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": temperature,
        "stream": False
    }
    
    resp = requests.post(endpoint, headers=headers, json=payload, timeout=(10, 120))
    resp.raise_for_status()
    data = resp.json()
    
    return data["message"]["content"]

In [ ]:
# Exercise 2: Implement get_response_text function

def get_response_text(prompt: str) -> str:
    """Call LLM and return just the response text."""
    try:
        return call_llm(prompt, temperature=0.0)
    except Exception:
        return ""

In [ ]:
# Exercise 3: Implement parse_json_response function

def parse_json_response(prompt: str) -> tuple:
    """Call LLM, attempt to parse response as JSON, return (success, result)."""
    raw = get_response_text(prompt)
    
    # Strip markdown code blocks
    cleaned = raw.strip()
    pattern = r'^```(?:json)?\s*\n?(.*?)\n?```$'
    match = re.match(pattern, cleaned, re.DOTALL | re.IGNORECASE)
    if match:
        cleaned = match.group(1).strip()
    
    try:
        return (True, json.loads(cleaned))
    except json.JSONDecodeError as e:
        return (False, f"JSONDecodeError: {e}")

In [ ]:
# Exercise 4: Implement compare_temperatures function

def compare_temperatures(prompt: str) -> dict:
    """Compare LLM responses at different temperatures."""
    low = call_llm(prompt, temperature=0.0)
    high = call_llm(prompt, temperature=0.8)
    
    return {
        "low_temp": low,
        "high_temp": high,
        "are_identical": low == high
    }

In [ ]:
# Exercise 5: Implement build_structured_prompt function

def build_structured_prompt(system_instruction: str, task: str, constraints: list) -> str:
    """Build a structured prompt with system, task, and constraints."""
    constraint_lines = "\n".join([f"- {c}" for c in constraints])
    
    return f"""SYSTEM:
{system_instruction}

TASK:
{task}

CONSTRAINTS:
{constraint_lines}"""